In [46]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

from sklearn.metrics import r2_score,mean_squared_error,mean_absolute_error
from sklearn.linear_model import LinearRegression,Ridge,Lasso
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from xgboost import XGBRegressor
# from catboost import CatBoostRegressor
from sklearn.model_selection import RandomizedSearchCV


ModuleNotFoundError: No module named 'catboost'

In [2]:
df=pd.read_csv("data/StudentsPerformance.csv")

In [3]:
df.head()

,gender,race/ethnicity,parental level of education,lunch,test preparation course,math score,reading score,writing score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [6]:
# Defining dependent and independent variable.
x=df.drop('math score',axis=1)
y=df['math score']

In [8]:
# various categories 
print("categories in 'gender' variable : ",df['gender'].unique())
print("categories in 'race/ethnicity' : ",df['race/ethnicity'].unique())
print("categories in 'parental level of education' : ",df['parental level of education'].unique())
print("categories in 'lunch' : ",df['lunch'].unique())
print("categories in 'test preparation course' : ",df['test preparation course'].unique())

categories in 'gender' variable :  ['female' 'male']
categories in 'race/ethnicity' :  ['group B' 'group C' 'group A' 'group D' 'group E']
categories in 'parental level of education' :  ["bachelor's degree" 'some college' "master's degree" "associate's degree"
 'high school' 'some high school']
categories in 'lunch' :  ['standard' 'free/reduced']
categories in 'test preparation course' :  ['none' 'completed']


## creating column transformer

In [29]:
numerical_feature=x.select_dtypes(exclude='object').columns
cat_features=x.select_dtypes(include='object').columns

from sklearn.preprocessing import StandardScaler,OneHotEncoder
from sklearn.compose import ColumnTransformer

numerical_transformer=StandardScaler()
oh_transformer=OneHotEncoder()
preprocessor=ColumnTransformer(
[
    ("OneHotEncoder",oh_transformer,cat_features),
    ("StandardScaler",numerical_transformer,numerical_feature),
]
)


In [39]:
x=preprocessor.fit_transform(x)

In [42]:
x.shape

(1000, 19)

## train test split

In [44]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=42)

## create a function to give performance metrics

In [45]:
def performace_metrics(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mse)
    r2=r2_score(true,predicted)
    return mae,mse,rmse,r2

In [66]:
models={
    "linear Regression":LinearRegression(),
    "Ridge":Ridge(),
    "Lasso":Lasso(),
    "K-Neighbours":KNeighborsRegressor(),
    "Decision Tree Regression":DecisionTreeRegressor(),
    "Random Forest Regression":RandomForestRegressor(),
    "AdaBoost Regression":AdaBoostRegressor(),
    "XGBoost Regressor":XGBRegressor(),
    # "CatBoost Regressor":CatBoostRegresssor()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(x_train,y_train)   #train model
    
    #predict
    y_train_pedict=model.predict(x_train)
    y_test_predict=model.predict(x_test)
    
    #performance metrics evaluation
    model_train_mae,model_train_mse,model_train_rmse,model_train_r2=performace_metrics(y_train,y_train_pedict)
    model_test_mae,model_test_mse,model_test_rmse,model_test_r2=performace_metrics(y_test,y_test_predict)
    
    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    print("Model Performance for Training set ")
    print("Mean Absolute Error:{:.4f}".format(model_train_mae))
    print("Mean Squared Error:{:.4f}".format(model_train_mse))
    print("Root Mean Squared Error:{:.4f}".format(model_train_rmse))
    print("R2_score:{:.4f}".format(model_train_r2))
    
    print("---------------------------------------------------------------")
    
    print("Model Performance for Testing set ")
    print("Mean Absolute Error:{:.4f}".format(model_test_mae))
    print("Mean Squared Error:{:.4f}".format(model_test_mse))
    print("Root Mean Squared Error:{:.4f}".format(model_test_rmse))
    print("R2_score:{:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)

    print('='*65)
    
    

linear Regression
Model Performance for Training set 
Mean Absolute Error:28.3445
Mean Squared Error:4.2691
Root Mean Squared Error:5.3240
R2_score:0.8743
---------------------------------------------------------------
Model Performance for Testing set 
Mean Absolute Error:28.9156
Mean Squared Error:4.2053
Root Mean Squared Error:5.3773
R2_score:0.8812
Ridge
Model Performance for Training set 
Mean Absolute Error:28.3378
Mean Squared Error:4.2650
Root Mean Squared Error:5.3233
R2_score:0.8743
---------------------------------------------------------------
Model Performance for Testing set 
Mean Absolute Error:29.0563
Mean Squared Error:4.2111
Root Mean Squared Error:5.3904
R2_score:0.8806
Lasso
Model Performance for Training set 
Mean Absolute Error:43.4784
Mean Squared Error:5.2063
Root Mean Squared Error:6.5938
R2_score:0.8071
---------------------------------------------------------------
Model Performance for Testing set 
Mean Absolute Error:42.5064
Mean Squared Error:5.1579
Root M

In [71]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=["models","r2_score"]).sort_values(by=["r2_score"],ascending=False)

,models,r2_score
0,linear Regression,0.881171
1,Ridge,0.880593
5,Random Forest Regression,0.851259
6,AdaBoost Regression,0.844712
7,XGBoost Regressor,0.827797
2,Lasso,0.825320
3,K-Neighbours,0.784518
4,Decision Tree Regression,0.746546


[0.8811710947341137,
 0.880593148502874,
 0.8253197323627852,
 0.7845181853747702,
 0.7465463732809947,
 0.8512589367859509,
 0.8447117686504847,
 0.8277965653186071]